#### Cleaning Players Data

In [23]:
import pandas as pd
import regex as re
from datetime import datetime    
from unidecode import unidecode

In [24]:
players = pd.read_csv("2020.csv", index_col = [0])

In [25]:
players["Year"] = 2020

In [26]:
#Reformatting how club are written
players["Club"] =[unidecode(w) for w in players["Club"]]

players["Player"] =[unidecode(w) for w in players["Player"]]

In [27]:
players["Player"] = [re.sub(' +', ' ', w) for w in players["Player"]]

#### Transforming Values

In [28]:
#Getting the number of days left on contract
Contract_end = []

for w in players["Contract"] : 
    try:
        length = datetime.strptime(str(w), '%b %d, %Y') - datetime(2022,7,1,0,0,0,0)
        Contract_end.append(length.days)  
    except (ValueError, AttributeError): 
        Contract_end.append("NA")

players["Contract"] = Contract_end

In [29]:
#Getting the Age
Age = []

for row in players["Date of Birth"] :
    try : 
        Age.append(re.search(r'\((.*?)\)',row[row.rindex(',')+1:]).group(1))
    except (ValueError, AttributeError): 
        Age.append("NA")
        
players["Date of Birth"] = Age

players = players.rename(columns={'Date of Birth': 'Age'})

In [30]:
#Number of days since they joined the club
Longetivity = []
for w in players["Join"] : 
    try:
        length = datetime(2022,7,1,0,0,0,0) - datetime.strptime(str(w), '%b %d, %Y')
        Longetivity.append(length.days) 
    except (ValueError, AttributeError): 
        Longetivity.append("NA")

players["Join"] = Longetivity

In [31]:
#Converting to numeric
players[['Age', 'Join', 'Contract']] = players[['Age', 'Join', 'Contract']].apply(pd.to_numeric, errors='coerce', axis=1)

In [32]:
#Transforming the market value column
def value_to_float(x):
    x = str(x).replace('€', '')
    if 'k' in x:
        return float(x.replace('k', '')) / 1000
    if 'm' in x:
        return float(x.replace('m', ''))
    return 0

In [33]:
#Formatting
players['Market Value'] = players['Market Value'].apply(value_to_float)

In [34]:
#In case of January transfer/ loan keep the original club
players = players.sort_values('Join', ascending=False).drop_duplicates('Player').sort_index()

In [35]:
players.loc[players["Player"] == "Lionel Messi"]

,Player Link,Player,Position,Age,Country,Join,Contract,Market Value,Club,Club Link,Season,Year
League,,,,,,,,,,,,
LaLiga,https://www.transfermarkt.co.uk/lionel-messi/p...,Lionel Messi,Right Winger,34.0,Argentina,6209.0,1279.0,80.0,FC Barcelona,https://www.transfermarkt.co.uk/fc-barcelona/s...,https://www.transfermarkt.co.uk/laliga/startse...,2020


#### Dropping Columns

In [36]:
#Contract Column dropped even though I could be kept however contract is new contract
players = players.drop([ "Season", "Club Link", "Contract"], axis=1)

#### Club Cleaning

In [37]:
players["Club"] = [l.replace('FC', '') for l in players["Club"]]

In [38]:
players["Club"] = [l.replace('Wolverhampton Wanderers', 'Wolves') for l in players["Club"]]

In [39]:
players["Club"] = [l.replace('Olympique', '') for l in players["Club"]]

In [40]:
players["Club"] = [l.replace('Hertha BSC', 'Hertha') for l in players["Club"]]

In [41]:
players

,Player Link,Player,Position,Age,Country,Join,Market Value,Club,Year
League,,,,,,,,,
Bundesliga,https://www.transfermarkt.co.uk/florian-muller...,Florian Muller,Goalkeeper,23.0,Germany,1826.0,8.000,1.FSV Mainz 05,2020
Bundesliga,https://www.transfermarkt.co.uk/edmond-tapsoba...,Edmond Tapsoba,Centre-Back,22.0,Burkina Faso,882.0,40.000,Bayer 04 Leverkusen,2020
Bundesliga,https://www.transfermarkt.co.uk/clinton-mola/p...,Clinton Mola,Centre-Back,20.0,England,882.0,1.000,VfB Stuttgart,2020
Bundesliga,https://www.transfermarkt.co.uk/matheus-cunha/...,Matheus Cunha,Centre-Forward,22.0,Brazil,882.0,30.000,Hertha,2020
Bundesliga,https://www.transfermarkt.co.uk/stefan-ilsanke...,Stefan Ilsanker,Defensive Midfield,32.0,Austria,882.0,1.700,Eintracht Frankfurt,2020
...,...,...,...,...,...,...,...,...,...
Süper Lig,https://www.transfermarkt.co.uk/enes-karakus/p...,Enes Karakus,Centre-Forward,20.0,Turkey,696.0,0.075,Basaksehir FK,2020
Süper Lig,https://www.transfermarkt.co.uk/abdulkadir-tas...,Abdulkadir Tasdan,Goalkeeper,18.0,Turkey,696.0,0.050,Kayserispor,2020
Süper Lig,https://www.transfermarkt.co.uk/adem-metin-tur...,Adem Metin Turk,Right Winger,17.0,Turkey,696.0,0.075,Antalyaspor,2020


In [42]:
players.to_csv('Players_2020.csv', encoding='utf-8')